In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib
import analyze # takes some time since inits hax
from channel_dict import channel_dict
from tqdm import tqdm
import datetime
import pickle
from scipy.stats import norm
import shutil

%matplotlib inline

# Job Submission

In [ ]:
from make_runlist_new import write_spe_lists

#dry run
write_spe_lists(write=False)

In [ ]:
#writes new runlists
written=write_spe_lists(write=True)
print(written)

### This will take some time to run as job submission takes several hours. Alternatively, in the terminal you can run ./submit_jobs.sh ./runlists/[runlist name] for a single runlist, or ./large_submission.sh to submit jobs for all runlists in ./runlists

In [ ]:
def submit_job(file):
    command = "./submit_jobs.sh %s" % file
    print(command)
    !{command}

In [ ]:
for f in written:
    submit_job(f)

# Acc Vs Time

### Calculating acceptance and error data

In [2]:
import hax
#hax already initiated when analyze is imported

from spe_acceptance import data_dir_base, rawdata_dir


def data_exists(run_number):
    data_path=os.path.join(data_dir_base, 'run_%05d.h5' %int(run_number))
    return os.path.exists(data_path)

def all_data_exists(runlist):
    return all([data_exists(run) for run in runlist])

def file_to_list(runlist_file):
    return [int(run) for run in runlist_file.split('.')[0].split('_')[1:4]]

def get_run_time(run):
    return hax.runs.datasets[hax.runs.datasets.number == run].start.values[0]
    
def find_file(run):
    return [f for f in os.listdir('runlists') if any(int(run)==elem for elem in file_to_list(f))]

#### Retrieves previously loaded data from pickle file

In [3]:
#all runlists
runlists = [f for f in os.listdir('./runlists')]
bottom_runs = []
accs = []

ch_acc_dict={}
ch_err_l={}
ch_err_u={}

occ_dict={}

upper_errs=[]
lower_errs=[]
errors = []

on_ch_dict={}

missing_runs = []
LED_off = []
resubmit_files = []

with open('/project/lgrandi/xenon1t/spe_acceptance/ch_data/ch_data.pkl', 'rb') as cd:
    if os.stat('/project/lgrandi/xenon1t/spe_acceptance/ch_data/ch_data.pkl').st_size==0:
        print('No data in pickle file')
        data=[]
    else:
        data=pickle.load(cd)
saved_runlists=[]


for cd in data:
    if len(data)==0:
        continue
        
    runlist = file_to_list(cd.runlist)
    if not all_data_exists(runlist):
        resubmit_files.append(cd.runlist)
        for r in runs:
            if not data_exists(r):
                missing_runs.append(r)
        continue    
          
    saved_runlists.append(cd.runlist)
    
    
    acc=cd.acc
    acc_errs_l=cd.acc_errs_l
    acc_errs_u=cd.acc_errs_u
    acc_sys=cd.acc_sys
    acc_stat=cd.acc_stat
    occ=cd.occ
    occ_sys=cd.occ_sys
    occ_stat=cd.occ_stat
    on_channels = cd.on_channels
    
    on_ch_dict[cd.runlist]=on_channels
      
    
    bottom_run = runlist[0]
    topbulk_run = runlist[1]
    topring_run = runlist[2]
    
    if len(on_channels) < 200:
        LED_off.append(cd.runlist)
    
    else:
        ch_acc_dict[cd.runlist]=acc
        occ_dict[cd.runlist]=occ
    
        on_acc=acc[on_channels]

        #make error dicts
        ch_err_l[cd.runlist]=acc_errs_l
        ch_err_u[cd.runlist]=acc_errs_u

        bottom_runs.append(bottom_run)
        accs.append(np.mean(on_acc))
        #avg err=sqrt(sum(channel errors**2)/# of channels)
        lower_errs.append(np.sqrt(np.sum(acc_errs_l[on_channels]**2)/len(on_channels)))
        upper_errs.append(np.sqrt(np.sum(acc_errs_u[on_channels]**2)/len(on_channels)))

print("Data already exists for: ", saved_runlists)

Data already exists for:  ['runlist_9549_9550_9551.txt', 'runlist_12587_12588_12589.txt', 'runlist_7350_7351_7352.txt', 'runlist_6909_6910_6911.txt', 'runlist_6550_6551_6552.txt', 'runlist_7067_7068_7069.txt', 'runlist_12768_12769_12770.txt', 'runlist_11429_11430_11431.txt', 'runlist_7447_7448_7449.txt', 'runlist_16984_16985_16986.txt', 'runlist_10696_10697_10698.txt', 'runlist_6957_6958_6959.txt', 'runlist_12046_12047_12048.txt', 'runlist_17454_17455_17456.txt', 'runlist_7968_7969_7970.txt', 'runlist_7646_7647_7648.txt', 'runlist_18562_18563_18564.txt', 'runlist_11625_11626_11627.txt', 'runlist_17998_17999_18000.txt', 'runlist_18383_18384_18385.txt', 'runlist_10133_10134_10135.txt', 'runlist_8448_8449_8450.txt', 'runlist_7758_7759_7760.txt', 'runlist_6938_6939_6940.txt', 'runlist_16343_16344_16345.txt', 'runlist_9604_9605_9606.txt', 'runlist_9033_9034_9036.txt', 'runlist_7485_7486_7487.txt', 'runlist_11069_11070_11071.txt', 'runlist_7569_7570_7571.txt', 'runlist_16701_16702_16703.txt'

In [ ]:
#just newest runlist

#newest_runlist= find_file(sorted(bottom_runs)[-1])[0]
#print("Newest Runlist: ", newest_runlist)
#runlist = file_to_list(newest_runlist)

#if str(newest_runlist) not in saved_runlists:
r_l=['runlist_18562_18563_18564.txt', 'runlist_18383_18384_18385.txt']
rl=['./runlists/runlist_18928_18929_18930.txt', './runlists/runlist_18846_18847_18848.txt', './runlists/runlist_18562_18563_18564.txt', './runlists/runlist_18383_18384_18385.txt']
for runlist in tqdm(runlists):
    runs=file_to_list(runlist)
    if not all_data_exists(runs):
        resubmit_files.append(runlist)
        for r in runs:
            if not data_exists(r):
                missing_runs.append(r)
        continue


    bottom_run = runs[0]
    topbulk_run = runs[1]
    topring_run = runs[2]

    thresholds = analyze.get_thresholds(analyze.find_regular_run(bottom_run))

    acc, acc_stat, acc_sys = analyze.acceptance_3runs(bottom_run, topbulk_run, topring_run, thresholds, val2corr2=6, space='amplitude')
    occ, occ_sys, occ_stat = analyze.occupancy_3runs(bottom_run, topbulk_run, topring_run, val2corr2=6, space='amplitude')

    on_channels = np.where(occ > 0.05)[0]
    
    on_ch_dict[runlist]=on_channels

    if len(on_channels) < 200:
        LED_off.append(runlist)
    
    else:
        on_acc = acc[on_channels]
        on_occ = occ[on_channels]

        ch_acc_dict[runlist]=acc
        occ_dict[runlist]=occ


        acc_errs_l = np.sqrt(acc_stat[0]**2+acc_sys**2)
        acc_errs_u = np.sqrt(acc_stat[1]**2+acc_sys**2)

        ch_err_l[runlist]=acc_errs_l
        ch_err_u[runlist]=acc_errs_u

        accs.append(np.mean(on_acc))
        lower_errs.append(np.sqrt(np.sum(acc_errs_l[on_channels]**2)/len(on_channels)))
        upper_errs.append(np.sqrt(np.sum(acc_errs_u[on_channels]**2)/len(on_channels)))
        bottom_runs.append(bottom_run)

        cd=analyze.ch_data(runlist, get_run_time(bottom_run), acc, acc_errs_l, acc_errs_u, acc_sys, acc_stat, occ, occ_sys, occ_stat, on_channels)
        data.append(cd)

errors=np.array([lower_errs, upper_errs])

#### Calculates acceptance, errors for new runlist

In [7]:
#just newest runlist

newest_runlist= find_file(sorted(bottom_runs)[-1])[0]
print("Newest Runlist: ", newest_runlist)
runlist = file_to_list(newest_runlist)

if str(newest_runlist) not in saved_runlists:
    if not all_data_exists(runlist):
        resubmit_files.append(newest_runlist)
        for r in runlist:
            if not data_exists(r):
                missing_runs.append(r)
            continue


    bottom_run = runlist[0]
    topbulk_run = runlist[1]
    topring_run = runlist[2]

    thresholds = analyze.get_thresholds(analyze.find_regular_run(bottom_run))

    acc, acc_stat, acc_sys = analyze.acceptance_3runs(bottom_run, topbulk_run, topring_run, thresholds)
    occ, occ_sys, occ_stat = analyze.occupancy_3runs(bottom_run, topbulk_run, topring_run)

    on_channels = np.where(occ > 0.05)[0]
    
    on_ch_dict[newest_runlist]=on_channels

    if len(on_channels) < 200:
        LED_off.append(newest_runlist)
        print("LED likely off for %s" %newest_runlist)
        
    else:
        on_acc = acc[on_channels]
        on_occ = occ[on_channels]

        ch_acc_dict[newest_runlist]=acc
        occ_dict[newest_runlist]=occ

        acc_errs_l = np.sqrt(acc_stat[0]**2+acc_sys**2)
        acc_errs_u = np.sqrt(acc_stat[1]**2+acc_sys**2)

        ch_err_l[newest_runlist]=acc_errs_l
        ch_err_u[newest_runlist]=acc_errs_u

        accs.append(np.mean(on_acc))
        lower_errs.append(np.sqrt(np.sum(acc_errs_l[on_channels]**2)/len(on_channels)))
        upper_errs.append(np.sqrt(np.sum(acc_errs_u[on_channels]**2)/len(on_channels)))
        bottom_runs.append(bottom_run)

        cd=analyze.ch_data(newest_runlist, get_run_time(bottom_run), acc, acc_errs_l, acc_errs_u, acc_sys, acc_stat, occ, occ_sys, occ_stat, on_channels)
        data.append(cd)

else:
    print("Data already in pickle file")

errors=np.array([lower_errs, upper_errs])

Newest Runlist:  runlist_18928_18929_18930.txt
Data already in pickle file


#### Dumps all data into pickle file, prints runs with LED off and runs with missing data, deletes raw data for processed runs

In [ ]:
from spe_acceptance import change_permissions

os.remove('/project/lgrandi/xenon1t/spe_acceptance/ch_data/ch_data.pkl')
with open('/project/lgrandi/xenon1t/spe_acceptance/ch_data/ch_data.pkl', 'wb') as cd:
    pickle.dump(data, cd)
change_permissions('/project/lgrandi/xenon1t/spe_acceptance/ch_data/ch_data.pkl')

In [ ]:
#Delete raw data after processed data is in the pickle file
from get_name import get_name
no_data=[]
for runlist in runlists:
    with open(os.path.join('./runlists', runlist)) as r:
        for run in r.readlines():
            name=get_name(int(run))
            raw_path=os.path.join(rawdata_dir, name)
            pro_path=os.path.join(data_dir_base, 'run_%d.h5' % int(run))
            if not os.path.exists(raw_path):
                no_data.append(run)
                continue
            if os.path.exists(raw_path):
                if os.path.exists(pro_path):
                    shutil.rmtree(raw_path)
                    print("Deleting raw data for: ", run, name)
    
print("These runs are missing data: ", missing_runs)
print("LED likely off for these files: ", LED_off)

#### Resubmits jobs for runs missing data

In [ ]:
for f in resubmit_files:
    runs=file_to_list(f)

    submit=os.path.join('./runlists', f)
    if runs[0]<6731:
        continue
    submit_job(submit)

#### Acceptance dataframe with channels as row index, runlists as column index

In [9]:
acc_df=pd.DataFrame(ch_acc_dict)
channels_df=pd.DataFrame({"channels": [i for i in range(0,248)]})
ch_df=pd.concat([acc_df, channels_df], axis=1)
ch_df.set_index('channels', inplace=True)
pd.set_option('display.max_rows', 500)
#ch_df.to_csv('~/SPE/SPE/new_data', columns=newest_runlist)

#### Lower, upper errors dataframes with channels as row index, runlist as column index

In [10]:
lerr_df=pd.DataFrame(ch_err_l)
ch_lerr_df=pd.concat([lerr_df, channels_df], axis=1)
ch_lerr_df.set_index('channels', inplace=True)
#ch_lerr_df.to_csv('~/SPE/SPE/new_data', columns=newest_runlist)

In [11]:
uerr_df=pd.DataFrame(ch_err_u)
ch_uerr_df=pd.concat([uerr_df, channels_df], axis=1)
ch_uerr_df.set_index('channels', inplace=True)
#ch_uerr_df.to_csv('~/SPE/SPE/new_data', columns=newest_runlist)

In [16]:
rl=['runlist_18928_18929_18930.txt', 'runlist_18846_18847_18848.txt', 
    'runlist_18562_18563_18564.txt', 'runlist_18383_18384_18385.txt']

with open('/project/lgrandi/xenon1t/spe_acceptance/ch_data/new_runs.txt', 'w') as nr:
    for runlist in rl:
        on_ch=on_ch_dict[runlist]
        nr.write(runlist + "\n")
        nr.write("Mean Acc: \n") 
        nr.write(str(np.mean(ch_df.loc[on_ch, runlist]))+"\n")
        nr.write("Acc by Ch: \n")
        for i in ch_df.loc[on_ch, runlist]:
            nr.writelines("%s\n" %str(i))
        nr.write("Mean Error [lower, upper]: \n")
        nr.write(str(np.sqrt(np.sum(ch_lerr_df.loc[on_ch, runlist]**2))/len(on_ch))+',')
        nr.write(str(np.sqrt(np.sum(ch_uerr_df.loc[on_ch,runlist]**2))/len(on_ch)))
        nr.write("\n Lower Error by Ch:")
        for i in ch_lerr_df.loc[on_ch, runlist]:
            nr.writelines("%s\n" %str(i))
        nr.write("Upper Error by Ch:") 
        for i in ch_uerr_df.loc[on_ch, runlist]:
            nr.writelines("%s\n" %str(i))
    nr.close()

## Newest Runlist

#### Finds newest runlist, gets acceptance and occupancy 

#### Channels that look off, note that this includes both channels that are actually off and channels that aren't performing properly

In [ ]:
#prints list of off pmts
new_on_ch=on_ch_dict[newest_runlist]
my_off = np.where(occ_dict[newest_runlist][new_on_ch] < 0.05)[0]
print("Channels that look off for %s: " %newest_runlist , my_off)

#### Acceptance histogram for one runlist

In [ ]:
#plot the acceptance of one runlist

plt.hist(ch_df.loc[new_on_ch,newest_runlist], bins=50, range=(0,1.1))
plt.title("Acceptance for %s " %newest_runlist)
plt.xlabel("SPE Acceptance")
plt.ylabel("Counts")
plt.xlim(0, 1.1)
plt.show()

#### Occupancy histogram for one runlist

In [ ]:
#plot the occupancy of one runlist
plt.hist(occ_dict[newest_runlist][on_ch_dict[newest_runlist]], bins=50, range=(0,1.1))
plt.title("Occupancy for %s " %newest_runlist)
plt.xlabel("Occupancy")
plt.ylabel("Counts")
plt.xlim(0,0.25)
plt.show()

#### Acceptance of all channels for the newest runlist

In [ ]:
#plot acceptance of each channel

plt.figure(figsize=(9,6))
plt.errorbar(new_on_ch, ch_df.loc[new_on_ch,newest_runlist], yerr=[ch_lerr_df.loc[new_on_ch, newest_runlist], ch_uerr_df.loc[new_on_ch,newest_runlist]], marker='.', linestyle='none')
plt.ylabel('Acceptance Fraction')
plt.xlabel('Channel #')
plt.ylim(0.5,1.2)
plt.title('Acc by Ch for %s' %newest_runlist)
plt.show()

#### Prints low acceptance channels

In [ ]:
#prints list of low acc ch
cha_dict_new={}

for ch, a in zip(new_on_ch, ch_df.loc[new_on_ch, newest_runlist]):
    cha_dict_new[a]=ch
    
new_low_acc_ch=[]
for a in ch_df.loc[new_on_ch, newest_runlist]:
    if a<0.5:
        new_low_acc_ch.append(cha_dict_new[a])
        
print('Low acc ch for %s: ' %newest_runlist, new_low_acc_ch)

#### Mean, Median, Errors for runlist

In [ ]:
#maybe including off channels in mean
acc_mean=np.nanmean(ch_df.loc[new_on_ch, newest_runlist])
acc_median=np.nanmedian(ch_df.loc[new_on_ch, newest_runlist])
lower_error=np.sqrt(np.nansum(ch_lerr_df.loc[new_on_ch, newest_runlist]**2)/len(new_on_ch))
upper_error=np.sqrt(np.nansum(ch_uerr_df.loc[new_on_ch, newest_runlist]**2)/len(new_on_ch))

print("Runlist: ", newest_runlist)
print("Mean Acceptance: ", acc_mean)
print("Median Acceptance: ", acc_median)
print("Errors [l,u]: ", [lower_error, upper_error])

## Average Acceptance per Runlist

In [ ]:
#plot acceptance vs time using full runlists

import matplotlib.dates as mdates
years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
fmt = mdates.DateFormatter('%Y-%m')

bottom_runs = np.array(bottom_runs)
dates = np.array([get_run_time(run) for run in bottom_runs])
accs = np.array(accs)

noisy_runs = [12768, 13837]
sr1 = np.where(bottom_runs > 6731)
sr1 = [i for i in sr1[0] if bottom_runs[i] not in noisy_runs]
plot_dates = dates[sr1]
plot_runs = bottom_runs[sr1]
plot_accs = accs[sr1]
plot_errors = [errors[0][sr1],errors[1][sr1]]

f, ax = plt.subplots(figsize=(12,8))
ax.errorbar(plot_dates, plot_accs, yerr=plot_errors, linestyle='None', marker='.')
ax.set_ylim(0.75, 1.0)
plt.xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=20)
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(fmt)
ax.grid()
ax.set_xlabel('date')
ax.set_ylabel('spe acceptance')
plt.title("Acceptance vs Time")
plt.savefig('new_moneyplot.png')
plt.show()

#### Mean, Median, Errors for all runlists

In [ ]:
#add table of mean/median w errors for time evo, print days that are outliers

evo_mean=np.nanmean(plot_accs)
evo_median=np.nanmedian(plot_accs)
evo_errs=[np.nanmean(plot_errors[0]), np.nanmean(plot_errors[1])]

evo_std=np.nanstd(plot_accs)

print("All Runlists")
print("Mean :", evo_mean)
print("Median :", evo_median)
print("Errors [l, u]: ", evo_errs)

## Acceptance for Individual Channels

### One Channel

#### Plots acceptances of one channel for all runlists

In [ ]:
ch=126

#list of lists of runlists
sr1_rl=[find_file(i) for i in bottom_runs[sr1]]

#list of runlists
sr1_rl=[runlist for elem in sr1_rl for runlist in elem]

plt.figure(figsize=(12,8))
plt.errorbar(plot_dates, ch_df.loc[ch, sr1_rl] , yerr=[ch_lerr_df.loc[ch, sr1_rl], ch_uerr_df.loc[ch, sr1_rl]], linestyle='None', marker='.')
plt.ylim(0.75, 1.1)
plt.xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=20)
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(fmt)
plt.grid()
plt.title("Channel %s" %ch)
plt.xlabel('date')
plt.ylabel('spe acceptance')
plt.show()

In [ ]:
#add table of mean/median w errors for time evo, print days that are outliers

ch_mean=np.nanmean(ch_df.loc[ch, sr1_rl])
ch_median=np.nanmedian(ch_df.loc[ch,sr1_rl])
ch_errs=[np.nanmean(ch_lerr_df.loc[ch, sr1_rl]), np.nanmean(ch_uerr_df.loc[ch, sr1_rl])]

ch_std=np.nanstd(ch_df.loc[ch,sr1_rl])

print("All Runlists for ch %d" %ch)
print("Mean :", ch_mean)
print("Median :", ch_median)
print("Errors [l, u]: ", ch_errs)

#### Prints runlists with low acceptance for one channel

In [ ]:
#prints list of low acc runlists

ch_accs=np.array(ch_df.loc[ch])

low_acc=np.where(ch_accs<0.5)

low_acc_rl=list(ch_df.columns[low_acc])
        
print('Low acc runlists for ch %s: ' %ch, low_acc_rl)

In [ ]:
import matplotlib.dates as mdates
from dateutil.relativedelta import relativedelta
import datetime

months = mdates.MonthLocator()
years = mdates.YearLocator()
datetimefmt = mdates.DateFormatter("%d-%m")


dates = [get_run_time(run) for run in bottom_runs]
today = datetime.date.today()

print(len(dates), np.shape(ch_df.loc[ch,:]), np.shape(ch_lerr_df.loc[ch, :]), np.shape(ch_uerr_df.loc[ch,:]))
def plot_channel(ch):
    f, ax = plt.subplots(figsize=(8,5))
    ax.fmt_xdata = mdates.DateFormatter('%d-%m')
    ax.errorbar(dates, ch_df.loc[ch,:], yerr=[ch_lerr_df.loc[ch, :], ch_uerr_df.loc[ch,:]], linestyle='None', color='b', marker='.')
    ax.xaxis.set_major_formatter(datetimefmt)
    ax.xaxis.set_major_locator(months)
    plt.xlabel('Date')
    plt.ylabel('SPE Acceptance')
    plt.title('Channel %d' %ch)
    f.autofmt_xdate()
    plt.grid()
    plt.ylim(-0.1, 1.1)
    plt.xlim(datetime.date(year=2018, month=1, day=1), datetime.date(year=2018, month=5, day=1) )
    #plt.savefig('plots/ch%d.png' % ch)
    plt.show()

In [ ]:
table = "^Channel^Acceptance^\n"

for ch in [12, 34, 86, 134, 148, 190, 213]:
    plot_channel(ch)
    indices = [dates.index(date) for date in sorted(dates)[-2:]]
    a = np.mean(ch_df.iloc[ch, indices])
    table += "|%d|%0.2f|\n" % (ch, a)

table2 = "^Channel^Acceptance^Uncertainty^\n"
for ch in [27, 73, 167, 203]:
    plot_channel(ch)
    indices = [dates.index(date) for date in sorted(dates)[-2:]]
    a = np.mean(ch_df.iloc[ch, indices])
    table2 += "|%d|%0.2f|\n" % (ch, a)

In [ ]:
print(table)

print(table2)

# Further Analysis

### Find outliers, plot the acceptance curve, acceptance histogram, and occupancy histogram

#### Change the index in run=outliers[index] in order to change which outlier gets plotted

In [ ]:
#Find outliers, plot acc of one of them

outlier_indices_low = np.array(np.where(accs < (evo_mean-1.5*evo_std))).flatten()
outlier_indices_high= np.array(np.where(accs > (evo_mean+1.5*evo_std))).flatten()

outlier_indices=outlier_indices_low.tolist() + outlier_indices_high.tolist()

outliers = bottom_runs[outlier_indices]
print("Outliers :", outliers)
run = outliers[0] #change index to change which outlier
print('Run: ',run)
runlist = file_to_list(find_file(run)[0])

bot_run=runlist[0]
tb_run=runlist[1]
tr_run=runlist[2]

x, acc, err = analyze.acceptance_curve_3runs(bot_run, tb_run, tr_run)

plt.figure(figsize=(9,6))
for ch, a in enumerate(acc):
    plt.plot(x, a, color='black', linewidth=1, alpha=0.6)
    plt.title("Acceptance curve for: %d" %run)
    
plt.xlim(-10, 200)
plt.xlabel('threshold')
plt.ylabel('spe acceptance')
plt.show()

In [ ]:
#doesn't save the data
thresholds = analyze.get_thresholds(analyze.find_regular_run(bot_run))
acc, acc_stat, acc_sys = analyze.acceptance_3runs(bot_run, tb_run, tr_run, thresholds)
occ, occ_sys, occ_stat = analyze.occupancy_3runs(bot_run, tb_run, tr_run)

on_channels = np.where(occ > 0.05)[0]

acc=acc[on_channels]
occ=occ[on_channels]

In [ ]:
#plot the occupancy of one runlist
plt.hist(occ, bins=50, range=(0,1.1))
plt.title("Occupancy for %s " %find_file(run)[0])
plt.xlabel("Occupancy")
plt.ylabel("Counts")
plt.xlim(0,0.25)
plt.show()

In [ ]:
#plot the acceptance of one runlist
plt.hist(acc, bins=50, range=(0,1.1))
plt.title("Acceptance for %s " %find_file(run)[0])
plt.xlabel("SPE Acceptance")
plt.ylabel("Counts")
plt.xlim(0, 1.1)
plt.show()

In [ ]:
runlist= find_file(run)[0]

#plot acceptance of each channel
plt.figure()
plt.figure(figsize=(12,8))
plt.errorbar(on_channels, acc, yerr=[ch_lerr_df.loc[on_channels, runlist], ch_uerr_df.loc[on_channels,runlist]], marker='.', linestyle='none')
plt.ylabel('Acceptance Fraction')
plt.xlabel('Channel #')
plt.title('Acc by Ch for %s' %runlist)
plt.show()

In [ ]:
cha_dict={}

for ch, a in zip(on_channels, acc):
    cha_dict[a]=ch
    
low_acc_ch=[]
for a in acc:
    if a<0.5:
        low_acc_ch.append(cha_dict[a])
        
print('Low acc ch: ', low_acc_ch)